In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
dataset_path = "/content/drive/MyDrive/Dataset_IC01/Gigi Tampak Depan"

In [ ]:
class_labels = ['Radang Gusi', 'Perubahan Warna Gigi', 'Gigi Sehat', 'Gigi berlubang', 'Bukan Gigi']

In [ ]:
import os
files = os.listdir(dataset_path)
print(files)

['Gigi Sehat', 'Gigi Berlubang', 'Perubahan Warna Gigi', 'Radang Gusi', 'Bukan Gigi']


In [ ]:
# Initialize an empty list to store the image files
image_files = []

# Iterate over the subdirectoriesa
for subdir in os.listdir(dataset_path):
    subdir_path = os.path.join(dataset_path, subdir)
    for file in os.listdir(subdir_path):
        if file.endswith(('.jpg', '.jpeg', '.png')):
            image_files.append(os.path.join(subdir_path, file))

In [ ]:
# Image dimensions
img_height, img_width = 224, 224
batch_size = 64

In [ ]:
# Prepare the dataset
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 6191 images belonging to 5 classes.
Found 1545 images belonging to 5 classes.


In [ ]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 222, 222, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 batch_normalization_1 (Bat  (None, 109, 109, 64)      256       
 chNormalization)                                                
                                                        

In [ ]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') >= 0.98 and logs.get('val_accuracy') >= 0.98:
            print("\nReached 98% accuracy and validation accuracy, stopping training.")
            self.model.stop_training = True

callbacks = [CustomCallback(), early_stopping, reduce_lr]

In [14]:
# Train the model
model.fit(
    train_generator,
    epochs=30,  # Set a high number to allow early stopping
    validation_data=validation_generator,
    callbacks=callbacks
)

Epoch 1/30
97/97 [==============================] - 3829s 39s/step - loss: 9.1032 - accuracy: 0.4933 - val_loss: 25.7602 - val_accuracy: 0.3282 - lr: 0.0010
Epoch 2/30
97/97 [==============================] - 1206s 12s/step - loss: 1.2984 - accuracy: 0.5484 - val_loss: 18.6162 - val_accuracy: 0.3547 - lr: 0.0010
Epoch 3/30
97/97 [==============================] - 1140s 12s/step - loss: 1.0682 - accuracy: 0.5829 - val_loss: 30.7823 - val_accuracy: 0.2828 - lr: 0.0010
Epoch 4/30
97/97 [==============================] - 1122s 12s/step - loss: 0.9465 - accuracy: 0.6330 - val_loss: 16.0473 - val_accuracy: 0.3398 - lr: 0.0010
Epoch 5/30
97/97 [==============================] - 1178s 12s/step - loss: 0.8851 - accuracy: 0.6611 - val_loss: 1.9675 - val_accuracy: 0.5812 - lr: 0.0010
Epoch 6/30
97/97 [==============================] - 1130s 12s/step - loss: 0.9958 - accuracy: 0.6595 - val_loss: 1.2515 - val_accuracy: 0.6207 - lr: 0.0010
Epoch 7/30
97/97 [==============================] - 1118s 12

In [15]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy:.2f}')

25/25 [==============================] - 83s 3s/step - loss: 0.8914 - accuracy: 0.6997
Validation accuracy: 0.70


In [16]:
# Save the model
model.save('cnn_image_classifier.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
